# Train YOLOv8n for Golf Ball Detection

This notebook fine-tunes YOLOv8 nano on golf ball images for use with OpenLaunch.

**Instructions:**
1. Open in Google Colab
2. Go to Runtime > Change runtime type > Select GPU (T4)
3. Run all cells
4. Download the trained model at the end

In [ ]:
# Install dependencies
!pip install ultralytics roboflow -q

In [ ]:
# Download golf ball dataset from Roboflow
from roboflow import Roboflow

# Using the Golf Ball and Hole Detection dataset (1,171 images)
# You can also try: https://universe.roboflow.com/appleroot-zk4px/golf-ball-detection-w0ixa
rf = Roboflow(api_key="YOUR_API_KEY")  # Get free API key at roboflow.com

# Option 1: Golf ball and hole detection (larger dataset)
project = rf.workspace("sai-gon-university").project("golf-ball-and-hole-detection-1k7")
dataset = project.version(1).download("yolov8")

# If you don't want to use an API key, you can manually download:
# 1. Go to https://universe.roboflow.com/sai-gon-university/golf-ball-and-hole-detection-1k7
# 2. Click "Download Dataset" > YOLOv8 format
# 3. Upload the zip to Colab and unzip

In [ ]:
# Alternative: Download without API key using direct link
# Uncomment if you prefer not to create a Roboflow account

# !mkdir -p datasets
# !curl -L "https://universe.roboflow.com/ds/DATASET_URL" > datasets/golfball.zip
# !unzip -q datasets/golfball.zip -d datasets/golfball

In [ ]:
from ultralytics import YOLO

# Load pretrained YOLOv8 nano
model = YOLO('yolov8n.pt')

# Check dataset path
import os
dataset_path = dataset.location if 'dataset' in dir() else 'datasets/golfball'
print(f"Dataset path: {dataset_path}")
print(f"Contents: {os.listdir(dataset_path)}")

In [ ]:
# Train the model
# This takes about 30-60 minutes on a T4 GPU

results = model.train(
    data=f"{dataset_path}/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    patience=20,  # Early stopping
    device=0,  # Use GPU
    workers=2,
    project="golf_ball_detector",
    name="yolov8n_golfball",
    pretrained=True,
    optimizer="AdamW",
    lr0=0.001,
    augment=True,
)

In [ ]:
# Evaluate the model
metrics = model.val()
print(f"\nmAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")

In [ ]:
# Export to NCNN format for Raspberry Pi
best_model = YOLO('golf_ball_detector/yolov8n_golfball/weights/best.pt')

# Export to NCNN (optimized for ARM/Pi)
best_model.export(format='ncnn', imgsz=640)

print("\nExported to NCNN format!")

In [ ]:
# Also export standard PyTorch for flexibility
!cp golf_ball_detector/yolov8n_golfball/weights/best.pt golf_ball_yolov8n.pt
print("Saved: golf_ball_yolov8n.pt")

In [ ]:
# Download the trained models
from google.colab import files

# Download PyTorch model
files.download('golf_ball_yolov8n.pt')

# Zip and download NCNN model
!zip -r golf_ball_ncnn.zip golf_ball_detector/yolov8n_golfball/weights/best_ncnn_model/
files.download('golf_ball_ncnn.zip')

## Usage on Raspberry Pi

After downloading, transfer to your Pi:

```bash
# Copy model to Pi
scp golf_ball_yolov8n.pt pi@<ip>:~/openlaunch/models/
scp golf_ball_ncnn.zip pi@<ip>:~/openlaunch/models/

# On Pi, unzip NCNN model
cd ~/openlaunch/models
unzip golf_ball_ncnn.zip

# Test it
cd ~/openlaunch
python scripts/test_yolo_detection.py --model models/golf_ball_yolov8n.pt
# Or with NCNN
python scripts/test_yolo_detection.py --model models/best_ncnn_model
```